In [2]:
!pip install kagglehub




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
   ---------------------------------------- 43.0/43.0 kB 1.0 MB/s eta 0:00:00


In [3]:
!pip show kagglehub


Name: kagglehub
Version: 0.3.4
Summary: Access Kaggle resources anywhere
Home-page: 
Author: 
Author-email: Kaggle <support@kaggle.com>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      other

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TensorFlow version:", tf.__version__)
print("Keras submodule is available.")


TensorFlow version: 2.18.0
Keras submodule is available.


In [2]:
pip install --upgrade pip



  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/ef/7d/500c9ad20238fcfcb4cb9243eede163594d7020ce87bd9610c9e02771876/pip-24.3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 656.4 kB/s eta 0:00:03
   ------ --------------------------------- 0.3/1.8 MB 2.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 3.7 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 6.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1


In [7]:
import os
print("Dataset path:", path)
print("Contents of the dataset path:", os.listdir(path))


Dataset path: C:\Users\s350\.cache\kagglehub\datasets\azzaali\updated-knee-osteoarthritis-dataset\versions\1
Contents of the dataset path: ['kaggle']


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set directory paths
train_dir = r"C:\Users\s350\OneDrive\Desktop\KNEE\archive (1)\kaggle\working\split_data\train"
val_dir = r"C:\Users\s350\OneDrive\Desktop\KNEE\archive (1)\kaggle\working\split_data\val"
test_dir = r"C:\Users\s350\OneDrive\Desktop\KNEE\archive (1)\kaggle\working\split_data\test"

# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"  # Change to "binary" if you have two classes
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"  # Change to "binary" if you have two classes
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"  # Change to "binary" if you have two classes
)

# Load MobileNetV2 as the base model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
x = Dense(128, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",  # Use "binary_crossentropy" for binary classification
              metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

# Save the model
model.save("knee_osteoarthritis_mobilenetv2.h5")

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Found 5806 images belonging to 5 classes.
Found 1452 images belonging to 5 classes.
Found 1526 images belonging to 5 classes.
Epoch 1/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3756 - loss: 1.4312

C:\Users\s350\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


182/182 ━━━━━━━━━━━━━━━━━━━━ 273s 1s/step - accuracy: 0.3757 - loss: 1.4304 - val_accuracy: 0.4360 - val_loss: 1.1545
Epoch 2/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.4257 - loss: 1.1685 - val_accuracy: 0.4614 - val_loss: 1.1181
Epoch 3/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 233s 1s/step - accuracy: 0.4428 - loss: 1.1328 - val_accuracy: 0.4814 - val_loss: 1.1148
Epoch 4/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.4448 - loss: 1.1332 - val_accuracy: 0.4800 - val_loss: 1.0904
Epoch 5/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.4651 - loss: 1.1070 - val_accuracy: 0.4931 - val_loss: 1.0797
Epoch 6/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.4649 - loss: 1.0931 - val_accuracy: 0.4959 - val_loss: 1.0776
Epoch 7/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.4798 - loss: 1.0772 - val_accuracy: 0.4745 - val_loss: 1.1022
Epoch 8/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 222s 1s/step - accuracy: 0.4699 - loss: 1.0993 - val_accuracy: 0.482

48/48 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.4915 - loss: 1.2862
Test Loss: 1.3058, Test Accuracy: 0.4830
